In [16]:
from opensearchpy import OpenSearch
from angle_emb import AnglE, Prompts

c:\Users\a.almasri\AppData\Local\Programs\Python\Python39\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'



In [18]:
# Initialize AnglE embedding model
angle = AnglE.from_pretrained('WhereIsAI/UAE-Large-V1', pooling_strategy='cls').cuda()
# Enable Prompt.C for retrieval optimized embeddings
angle.set_prompt(prompt=Prompts.C)   

INFO:AnglE:Prompt is set, the prompt will be automatically applied during the encoding phase. To disable prompt setting, please configure set_prompt(prompt=None)


In [19]:
# OpenSearch instance parameters
host = 'localhost'
port = 9200
auth = ('admin', 'admin')

# Create the client with SSL/TLS enabled and disable warnings
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],    
    http_auth = auth,
    use_ssl = True,
    verify_certs = False,
    ssl_show_warn = False,
)

In [20]:
# Testing connection to OpenSearch
client.info()

INFO:opensearch:GET https://localhost:9200/ [status:200 request:0.339s]


{'name': 'opensearch',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'ghgK1h_FSeefVD9yf6_gag',
 'version': {'distribution': 'opensearch',
  'number': '2.11.1',
  'build_type': 'tar',
  'build_hash': '6b1986e964d440be9137eba1413015c31c5a7752',
  'build_date': '2023-11-29T21:43:10.135035992Z',
  'build_snapshot': False,
  'lucene_version': '9.7.0',
  'minimum_wire_compatibility_version': '7.10.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'The OpenSearch Project: https://opensearch.org/'}

In [24]:
# Query embedding
query = "where does hypotonia typically appear?"
query_emb = angle.encode({'text': query})

In [27]:
# Defining the knn query parameters
search_query = {    
    "query": {
        "knn": {
            "embedding": {
                "vector": query_emb[0].tolist(),
                "k": 3
            }
        }
    }
}

In [28]:
# Send a knn query to OpenSearch
results = client.search(index="pubmed_index", body=search_query)

INFO:opensearch:POST https://localhost:9200/pubmed_index/_search [status:200 request:0.162s]


In [29]:
# Helper function to view the reposes easily
def pretty_response(response):
    for hit in response['hits']['hits']:
        id = hit['_id']
        score = hit['_score']
        pmid = hit['_source']['pmid']
        chunk_id = hit['_source']['chunk_id']  
        chunk = hit['_source']['chunk']      
        pretty_output = (f"\nID: {id}\nPMID: {pmid}\nChunk ID: {chunk_id}\nText: {chunk}")
        print(pretty_output)

In [30]:
pretty_response(results)


ID: HAsgOI0BrWW60NSubi17
PMID: 23333410
Chunk ID: 6
Text: VIQ score in the severe delay range. Abnormal muscle tonicity was found in 35% (hypotonicity 33%, hypertonicity 2%). Need for ECMO, prolonged ventilation, hypotonicity, and other surrogate markers of disease severity (P<0.05) were associated with borderline or delayed neurological outcome. CONCLUSION: The majority of CDH children are functioning in the average range at early preschool and preschool age. Neuromuscular hypotonicity is common in CDH survivors. CDH severity appears to be predictive

ID: hgsgOI0BrWW60NSuZys6
PMID: 20301331
Chunk ID: 1
Text: In infancy, hypotonia is typical, and acquisition of developmental motor milestones is often both aberrant in pattern and delayed. Intelligence and life span are usually near normal, although craniocervical junction compression increases the risk of death in infancy. Additional complications include obstructive sleep apnea, middle ear dysfunction, kyphosis, and spinal stenosis. D